In [1]:
import pandas as pd

# Read without headers
df = pd.read_csv("captures_combined.csv", header=None)

df.columns = [
    "timestamp",
    "client_ip",
    "method",
    "url",
    "path",
    "body",
    "body_len",
    "body_entropy",
    "special_char_count",
    "path_depth",
    "query_length",
    "has_file_upload",
    "response_status",
    "response_length"
]

# Save fixed file
df.to_csv("captures_combined_fixed.csv", index=False)

print("CSV header fixed successfully")


CSV header fixed successfully


In [7]:
import pandas as pd

df = pd.read_csv("captures_combined_fixed.csv")

def label_row(row):
    text = (str(row["url"]) + " " + str(row["body"])).lower()

    # SQL Injection (strict)
    if any(x in text for x in [
        "union select",
        "select ",
        "extractvalue",
        "updatexml",
        "' or",
        " or 1=1",
        "--"
    ]):
        return 1

    # XSS (decode-aware patterns)
    if any(x in text for x in [
        "<script",
        "%3cscript",
        "%3c%73%63%72%69%70%74",
        "onerror",
        "onload",
        "alert("
    ]):
        return 2

    # Command Injection (OS-level only)
    if any(x in text for x in [
        "&&",
        "||",
        "%0a",
        "%0d",
        "whoami",
        "uname",
        "id",
        "cat ",
        "/bin/"
    ]):
        return 3

    # Directory Traversal
    if any(x in text for x in [
        "../",
        "..%2f",
        "%2e%2e%2f",
        "etc/passwd",
        "proc/self"
    ]):
        return 4

    # File Upload
    if row["has_file_upload"] == 1:
        return 5

    return 0



df["label"] = df.apply(label_row, axis=1)

df.to_csv("labeled_attacks.csv", index=False)
print("Labeling complete")


Labeling complete


In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv("labeled_attacks.csv")
df.head(50)

,timestamp,client_ip,method,url,path,body,body_len,body_entropy,special_char_count,path_depth,query_length,has_file_upload,response_status,response_length,label
0,2025-12-22T15:58:09.876363Z,"('127.0.0.1', 57812)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a&password=a,21,3.499,1,2,0,0,200,1289,0
1,2025-12-22T15:58:11.745815Z,"('127.0.0.1', 43782)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a&password=a,21,3.499,1,2,0,0,200,1289,0
2,2025-12-22T15:58:11.857624Z,"('127.0.0.1', 43798)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=5305&password=a,24,3.855,1,2,0,0,200,1289,0
3,2025-12-22T15:58:11.905631Z,"('127.0.0.1', 43812)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%29%22%28%27%28%2C%28.%2C.&password=a,47,3.852,1,2,0,0,200,1289,0
4,2025-12-22T15:58:11.948084Z,"('127.0.0.1', 43820)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%27NAQqov%3C%27%22%3ErWrBJX&password=a,48,4.536,1,2,0,0,200,1289,0
5,2025-12-22T15:58:12.018931Z,"('127.0.0.1', 43830)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%27%29%20AND%205831%3D5262%20AND%20%...,79,4.411,1,2,0,0,200,1289,0
6,2025-12-22T15:58:12.056731Z,"('127.0.0.1', 43834)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%27%20AND%208330%3D3785%20AND%20%27x...,73,4.316,1,2,0,0,200,1289,0
7,2025-12-22T15:58:12.096664Z,"('127.0.0.1', 43848)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%29%20AND%201623%3D1951%20AND%20%287...,67,4.362,1,2,0,0,200,1289,0
8,2025-12-22T15:58:12.132793Z,"('127.0.0.1', 43856)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%20AND%206885%3D5951&password=a,41,4.473,1,2,0,0,200,1289,0
9,2025-12-22T15:58:12.173733Z,"('127.0.0.1', 43862)",POST,http://192.168.122.170/dvwa/login.php,/dvwa/login.php,username=a%20AND%203997%3D7036--%20wZJO&passwo...,50,4.559,1,2,0,0,200,1289,1
